In [1]:
# Run the following to sync data
#!mkdir ~/GitHub/junk-drawer-analyses/geneseek_data
#!aws s3 sync s3://legx-geneseek/technical-variation-projects/working-data/ ~/GitHub/junk-drawer-analyses/geneseek_data


# Import modules

In [2]:
from methylprep import run_pipeline
from scipy import stats
from statsmodels.distributions.empirical_distribution import ECDF
import pandas as pd
import numpy as np

# Define functions

In [3]:
def pval_minfi(data_containers):
    
    # Pull M and U values
    meth = pd.DataFrame(data_containers[0]._SampleDataContainer__data_frame.index)
    unmeth = pd.DataFrame(data_containers[0]._SampleDataContainer__data_frame.index)

    for i,c in enumerate(data_containers):
        sample = data_containers[i].sample
        m = c._SampleDataContainer__data_frame.rename(columns={'meth':sample})
        u = c._SampleDataContainer__data_frame.rename(columns={'unmeth':sample})
        meth = pd.merge(left=meth,right=m[sample],left_on='IlmnID',right_on='IlmnID',)
        unmeth = pd.merge(left=unmeth,right=u[sample],left_on='IlmnID',right_on='IlmnID')
    
    # Create empty dataframes for red and green negative controls
    negctlsR = pd.DataFrame(data_containers[0].ctrl_red['Extended_Type'])
    negctlsG = pd.DataFrame(data_containers[0].ctrl_green['Extended_Type'])

    # Fill red and green dataframes
    for i,c in enumerate(data_containers):
        sample = str(data_containers[i].sample)
        dfR = c.ctrl_red
        dfR = dfR[dfR['Control_Type']=='NEGATIVE']
        dfR = dfR[['Extended_Type','mean_value']].rename(columns={'mean_value':sample})
        dfG = c.ctrl_green
        dfG = dfG[dfG['Control_Type']=='NEGATIVE']
        dfG = dfG[['Extended_Type','mean_value']].rename(columns={'mean_value':sample})
        negctlsR = pd.merge(left=negctlsR,right=dfR,on='Extended_Type')
        negctlsG = pd.merge(left=negctlsG,right=dfG,on='Extended_Type')

    # Reset index on dataframes    
    negctlsG = negctlsG.set_index('Extended_Type')
    negctlsR = negctlsR.set_index('Extended_Type')
    
    # Get M and U values for IG, IR and II

    # first pull out sections of manifest (will be used to identify which probes belong to each IG, IR, II)
    manifest = data_containers[0].manifest.data_frame[['Infinium_Design_Type','Color_Channel']]
    IG = manifest[(manifest['Color_Channel']=='Grn') & (manifest['Infinium_Design_Type']=='I')]
    IR = manifest[(manifest['Color_Channel']=='Red') & (manifest['Infinium_Design_Type']=='I')]
    II = manifest[manifest['Infinium_Design_Type']=='II']

    # second merge with meth and unmeth dataframes
    IG_meth = pd.merge(left=IG,right=meth,on='IlmnID').drop(columns=['Infinium_Design_Type','Color_Channel']).set_index('IlmnID')
    IG_unmeth = pd.merge(left=IG,right=unmeth,on='IlmnID').drop(columns=['Infinium_Design_Type','Color_Channel']).set_index('IlmnID')
    IR_meth = pd.merge(left=IR,right=meth,on='IlmnID').drop(columns=['Infinium_Design_Type','Color_Channel']).set_index('IlmnID')
    IR_unmeth = pd.merge(left=IR,right=unmeth,on='IlmnID').drop(columns=['Infinium_Design_Type','Color_Channel']).set_index('IlmnID')
    II_meth = pd.merge(left=II,right=meth,on='IlmnID').drop(columns=['Infinium_Design_Type','Color_Channel']).set_index('IlmnID')
    II_unmeth = pd.merge(left=II,right=unmeth,on='IlmnID').drop(columns=['Infinium_Design_Type','Color_Channel']).set_index('IlmnID')
    
    # Calcuate parameters
    sdG = stats.median_absolute_deviation(negctlsG)
    muG = np.median(negctlsG,axis=0)
    sdR = stats.median_absolute_deviation(negctlsR)
    muR = np.median(negctlsR,axis=0)
    
    # calculate p values for type 1 Red
    pIR = pd.DataFrame(index=IR_meth.index,
                   data=1 - stats.norm.cdf(IR_meth+IR_unmeth,2*muR,2*sdR),
                   columns=IR_meth.columns)

    # calculate p values for type 1 Green
    pIG = pd.DataFrame(index=IG_meth.index,
                   data=1 - stats.norm.cdf(IG_meth+IG_unmeth,2*muG,2*sdG),
                   columns=IG_meth.columns)

    # calculat4e p values for type II
    pII = pd.DataFrame(index=II_meth.index,
                  data=1-stats.norm.cdf(II_meth+II_unmeth,muR+muG,sdR+sdG),
                  columns=II_meth.columns)
    # concat and sort
    pval = pd.concat([pIR, pIG, pII])
    pval = pval.sort_values(by='IlmnID')
    return pval



In [4]:
def pval_sesame(data_containers):
    # Pull M and U values
    meth = pd.DataFrame(data_containers[0]._SampleDataContainer__data_frame.index)
    unmeth = pd.DataFrame(data_containers[0]._SampleDataContainer__data_frame.index)

    for i,c in enumerate(data_containers):
        sample = data_containers[i].sample
        m = c._SampleDataContainer__data_frame.rename(columns={'meth':sample})
        u = c._SampleDataContainer__data_frame.rename(columns={'unmeth':sample})
        meth = pd.merge(left=meth,right=m[sample],left_on='IlmnID',right_on='IlmnID',)
        unmeth = pd.merge(left=unmeth,right=u[sample],left_on='IlmnID',right_on='IlmnID')
        
    # Separate M and U values for IG, IR and II
    # first pull out sections of manifest (will be used to identify which probes belong to each IG, IR, II)
    manifest = data_containers[0].manifest.data_frame[['Infinium_Design_Type','Color_Channel']]
    IG = manifest[(manifest['Color_Channel']=='Grn') & (manifest['Infinium_Design_Type']=='I')]
    IR = manifest[(manifest['Color_Channel']=='Red') & (manifest['Infinium_Design_Type']=='I')]
    II = manifest[manifest['Infinium_Design_Type']=='II']

    # second merge with meth and unmeth dataframes
    IG_meth = pd.merge(left=IG,right=meth,on='IlmnID').drop(columns=['Infinium_Design_Type','Color_Channel']).set_index('IlmnID')
    IG_unmeth = pd.merge(left=IG,right=unmeth,on='IlmnID').drop(columns=['Infinium_Design_Type','Color_Channel']).set_index('IlmnID')
    IR_meth = pd.merge(left=IR,right=meth,on='IlmnID').drop(columns=['Infinium_Design_Type','Color_Channel']).set_index('IlmnID')
    IR_unmeth = pd.merge(left=IR,right=unmeth,on='IlmnID').drop(columns=['Infinium_Design_Type','Color_Channel']).set_index('IlmnID')
    II_meth = pd.merge(left=II,right=meth,on='IlmnID').drop(columns=['Infinium_Design_Type','Color_Channel']).set_index('IlmnID')
    II_unmeth = pd.merge(left=II,right=unmeth,on='IlmnID').drop(columns=['Infinium_Design_Type','Color_Channel']).set_index('IlmnID')
    
    pval = pd.DataFrame(data=manifest.index,columns=['IlmnID'])
    for i,c in enumerate(data_containers):
        funcG = ECDF(data_containers[i].oob_green['mean_value'].values)
        funcR = ECDF(data_containers[i].oob_red['mean_value'].values)
        sample = data_containers[i].sample
        pIR = pd.DataFrame(index=IR_meth.index,data=1-np.maximum(funcR(IR_meth[sample]), funcR(IR_unmeth[sample])),columns=[sample])
        pIG = pd.DataFrame(index=IG_meth.index,data=1-np.maximum(funcG(IG_meth[sample]), funcG(IG_unmeth[sample])),columns=[sample])
        pII = pd.DataFrame(index=II_meth.index,data=1-np.maximum(funcG(II_meth[sample]), funcR(II_unmeth[sample])),columns=[sample])
        p = pd.concat([pIR,pIG,pII]).reset_index()
        pval = pd.merge(pval,p)
    return pval

# Run pipeline & calculate p-values

## Project 1

In [5]:
# save beta values
project_data_containers = run_pipeline(data_dir='../geneseek_data/Geneseek_HUMMETV01_20190714/',
                                        betas=True,
                                        sample_sheet_filepath='../geneseek_data/Geneseek_HUMMETV01_20190714/samplesheet.csv',
                                   )
!mv beta_values.pkl ../geneseek_data/Geneseek_HUMMETV01_20190714/


100%|██████████| 8/8 [02:24<00:00, 18.09s/it]


In [6]:
# get uncorrected values
project_data_containers = run_pipeline(data_dir='../geneseek_data/Geneseek_HUMMETV01_20190714/',
                                        save_uncorrected=True,
                                        sample_sheet_filepath='../geneseek_data/Geneseek_HUMMETV01_20190714/samplesheet.csv',
                                   )


100%|██████████| 8/8 [02:31<00:00, 19.02s/it]


In [7]:
pval_minfi_df = pval_minfi(project_data_containers)
pval_sesame_df = pval_sesame(project_data_containers)
pval_minfi_df.to_csv('../geneseek_data/Geneseek_HUMMETV01_20190714/pvals_minfi.csv',index=True,header=True)
pval_sesame_df.to_csv('../geneseek_data/Geneseek_HUMMETV01_20190714/pvals_sesame.csv',index=True,header=True)

//anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
//anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
//anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


## Project 2

In [8]:
# save beta values
project_data_containers = run_pipeline(data_dir='../geneseek_data/Geneseek_RND_HUMMETV01_20190923/',
                                        betas=True,
                                        sample_sheet_filepath='../geneseek_data/Geneseek_RND_HUMMETV01_20190923/samplesheet.csv')

!mv beta_values* ../geneseek_data/Geneseek_RND_HUMMETV01_20190923/


100%|██████████| 120/120 [37:38<00:00, 20.49s/it]


In [9]:
# get uncorrected values
project_data_containers = run_pipeline(data_dir='../geneseek_data/Geneseek_RND_HUMMETV01_20190923/',
                                        save_uncorrected=True,
                                        sample_sheet_filepath='../geneseek_data/Geneseek_RND_HUMMETV01_20190923/samplesheet.csv')


100%|██████████| 120/120 [39:56<00:00, 24.27s/it]


In [10]:
pval_minfi_df = pval_minfi(project_data_containers)
pval_sesame_df = pval_sesame(project_data_containers)
pval_minfi_df.to_csv('../geneseek_data/Geneseek_RND_HUMMETV01_20190923/pvals_minfi.csv',index=True,header=True)
pval_sesame_df.to_csv('../geneseek_data/Geneseek_RND_HUMMETV01_20190923/pvals_sesame.csv',index=True,header=True)

## Project 3

In [11]:
project_data_containers = run_pipeline(data_dir='../geneseek_data/Geneseek_HUMMETV01_20190903/',
                                        betas=True,
                                        sample_sheet_filepath='../geneseek_data/Geneseek_HUMMETV01_20190903/samplesheet.csv')

!mv beta_values* ../geneseek_data/Geneseek_HUMMETV01_20190903/


100%|██████████| 8/8 [02:22<00:00, 17.83s/it]


In [12]:
project_data_containers = run_pipeline(data_dir='../geneseek_data/Geneseek_HUMMETV01_20190903/',
                                        save_uncorrected=True,
                                        sample_sheet_filepath='../geneseek_data/Geneseek_HUMMETV01_20190903/samplesheet.csv')



100%|██████████| 8/8 [02:34<00:00, 19.20s/it]


In [13]:
pval_minfi_df = pval_minfi(project_data_containers)
pval_sesame_df = pval_sesame(project_data_containers)
pval_minfi_df.to_csv('../geneseek_data/Geneseek_HUMMETV01_20190903/pvals_minfi.csv',index=True,header=True)
pval_sesame_df.to_csv('../geneseek_data/Geneseek_HUMMETV01_20190903/pvals_sesame.csv',index=True,header=True)

## Project 4

In [14]:
project_data_containers = run_pipeline(data_dir = '../geneseek_data/Geneseek_RND_2_HUMMETV01_20191014/',
                                       betas=True,
                                       sample_sheet_filepath='../geneseek_data/Geneseek_RND_2_HUMMETV01_20191014/samplesheet.csv')

!mv beta_values* ../geneseek_data/Geneseek_RND_2_HUMMETV01_20191014/                                       
    

100%|██████████| 16/16 [04:51<00:00, 18.38s/it]


In [15]:
project_data_containers = run_pipeline(data_dir = '../geneseek_data/Geneseek_RND_2_HUMMETV01_20191014/',
                                       save_uncorrected=True,
                                       sample_sheet_filepath='../geneseek_data/Geneseek_RND_2_HUMMETV01_20191014/samplesheet.csv')



100%|██████████| 16/16 [05:04<00:00, 19.05s/it]


In [17]:
pval_minfi_df = pval_minfi(project_data_containers)
pval_sesame_df = pval_sesame(project_data_containers)
pval_minfi_df.to_csv('../geneseek_data/Geneseek_RND_2_HUMMETV01_20191014/pvals_minfi.csv',index=True,header=True)
pval_sesame_df.to_csv('../geneseek_data/Geneseek_RND_2_HUMMETV01_20191014/pvals_sesame.csv',index=True,header=True)